In [20]:
### imports
import json
from nltk.tokenize import sent_tokenize, TweetTokenizer
from string import punctuation
import re
import regex
tokenizer = TweetTokenizer()
from nltk.corpus import stopwords
stopW = stopwords.words('english')
#nltk.download('stopwords')

In [21]:
### a method to Read a json file with img captions and article titles and parse it into set of documents
    
def getDocs(sourcePath):
    medDocs = []
    sourceFile = open(sourcePath, 'r')

    count = 1
    for line in sourceFile:
        medDocs.append(json.loads(line))
        count += 1

    sourceFile.close()
    
    # for k in range(1,count):
    #     print(str(k) + " : " + medDocs[k]['docId'])
    return medDocs


sourcePath = 'assets/medImages.jl'
medDocs = getDocs(sourcePath)
medDocs[262]

{'articleId': 'PMC7282154',
 'image_path': 'PMC7282154-6_i2376-0605-6-2-e50-f06.jpg',
 'image_url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/bin/i2376-0605-6-2-e50-f06.jpg',
 'articleUrl': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/',
 'articleTitle': 'RECURRENT INVASIVE DUCTAL BREAST CARCINOMA PRESENTING AS PRIMARY ADRENAL INSUFFICIENCY WITH ADRENAL CRISIS',
 'caption': 'Histologic section of adrenal metastatic disease (hematoxylin and eosin, Г—100). Tumor cells arranged in solid nests infiltrating lymph node tissue.'}

In [1]:
# method that cleans the title and caption text in each document
# the result is tokenized titles and caption without unwanted brackets, numbers, spaces, tags, unicode charts;
# result includes no punctuation and stopwards and tokens with length less than 2 symbols
# all tokens are lower case


from nltk.corpus import stopwords
stopW = stopwords.words('english')

def preprocessContent(content):
    sentences = sent_tokenize(content)
    tokens = []
    for sent in sentences:
        # removes unicode
        cleanedTokens = re.sub(r'[^\x00-\x7F]+', ' ', sent)
        # removes menions
        cleanedTokens = re.sub(r'@\w+', '', cleanedTokens)
        # removes numbers
        cleanedTokens = re.sub('[0-9]+', '', cleanedTokens)
        # removes nested brackets 
        cleanedTokens = regex.sub(r'\([^()]*+(?:(?R)[^()]*)*+\)', '', cleanedTokens) 
        # removes nested currly brackets
        cleanedTokens = regex.sub('\{(?:[^}{]|\{[^}{]*\})*\}', '', cleanedTokens) 
        # removes html tags
        cleanedTokens = re.sub('\<+/*\w*/*\>+', '', cleanedTokens)
        # removes punctuation
        cleanedTokens = re.sub(r'[%s]' % re.escape(punctuation), ' ', cleanedTokens)
        # removes doubled spaces
        cleanedTokens = re.sub(r'\s{2,}', ' ', cleanedTokens)
        sentTokens = tokenizer.tokenize(cleanedTokens)
        sentTokens = [tok.lower() for tok in sentTokens if not tok in stopW and len(tok) > 2]
        
        tokens += sentTokens
    return tokens


def cleanDocs(medDocs):
    cleanedDocs = medDocs
    for i, doc in enumerate(cleanedDocs):
        doc['articleTitle'] = preprocessContent(doc['articleTitle'] )
        doc['caption'] = preprocessContent(doc['caption'])

    return cleanedDocs

cleanedDocs = cleanDocs(medDocs)
print(len(cleanedDocs))
cleanedDocs[262]


NameError: name 'medDocs' is not defined

In [24]:
# constructing inverted index
def getInvertedIdx(medDocs):
    invertedIdx = {}

    for i, doc in enumerate(medDocs):
        for term in doc['articleTitle']:
            if term in invertedIdx:
                invertedIdx[term].append((i,'t'))
            else:
                invertedIdx[term] = [(i,'t')]
        for term in doc['caption']:
            if term in invertedIdx:
                invertedIdx[term].append((i,'c'))
            else:
                invertedIdx[term] = [(i,'c')]
    return invertedIdx
           
termInvIdx = getInvertedIdx(cleanedDocs)
termInvIdx


{'the': [(0, 't'),
  (0, 'c'),
  (1, 't'),
  (16, 't'),
  (17, 'c'),
  (17, 'c'),
  (23, 'c'),
  (33, 'c'),
  (33, 'c'),
  (33, 'c'),
  (36, 'c'),
  (40, 'c'),
  (41, 'c'),
  (41, 'c'),
  (42, 'c'),
  (43, 'c'),
  (44, 'c'),
  (46, 'c'),
  (52, 'c'),
  (59, 'c'),
  (60, 'c'),
  (62, 'c'),
  (63, 'c'),
  (64, 't'),
  (64, 'c'),
  (65, 't'),
  (65, 'c'),
  (66, 't'),
  (66, 'c'),
  (67, 'c'),
  (67, 'c'),
  (67, 'c'),
  (67, 'c'),
  (68, 'c'),
  (68, 'c'),
  (68, 'c'),
  (69, 'c'),
  (69, 'c'),
  (70, 'c'),
  (72, 'c'),
  (72, 'c'),
  (76, 'c'),
  (81, 'c'),
  (82, 'c'),
  (85, 'c'),
  (86, 'c'),
  (86, 'c'),
  (86, 'c'),
  (93, 'c'),
  (94, 'c'),
  (94, 'c'),
  (94, 'c'),
  (94, 'c'),
  (94, 'c'),
  (96, 'c'),
  (102, 'c'),
  (103, 'c'),
  (107, 'c'),
  (107, 'c'),
  (107, 'c'),
  (107, 'c'),
  (108, 'c'),
  (109, 'c'),
  (109, 'c'),
  (109, 'c'),
  (109, 'c'),
  (110, 'c'),
  (110, 'c'),
  (111, 'c'),
  (111, 'c'),
  (111, 'c'),
  (111, 'c'),
  (111, 'c'),
  (111, 'c'),
  (111, 'c'),
 

In [25]:
# sort the dict
sortedTermsPostings = {}
sortedTermsPostings = sorted(termInvIdx.items(),key=lambda x: x[0])

sortedTermsPostings = {k:v for k,v in sortedTermsPostings}

sortedTermsPostings


{'aaa': [(84, 'c'), (85, 'c'), (86, 'c'), (86, 'c')],
 'abbreviations': [(243, 'c')],
 'abdomen': [(0, 'c'), (4, 'c'), (253, 'c'), (253, 'c'), (253, 'c')],
 'abdominal': [(29, 'c')],
 'abel': [(165, 'c')],
 'ability': [(46, 'c'), (47, 'c'), (79, 'c')],
 'able': [(86, 'c')],
 'abnormal': [(252, 'c')],
 'abnormalities': [(252, 'c'), (252, 'c')],
 'abnormality': [(0, 'c')],
 'abs': [(9, 'c'), (111, 'c'), (111, 'c')],
 'absence': [(22, 'c'), (35, 'c'), (174, 'c'), (175, 'c')],
 'absolute': [(44, 'c')],
 'absolutely': [(110, 'c')],
 'absorbed': [(237, 'c')],
 'absorptiometry': [(252, 'c')],
 'absorption': [(64, 't'),
  (65, 't'),
  (66, 't'),
  (104, 'c'),
  (104, 'c'),
  (104, 'c'),
  (104, 'c'),
  (107, 'c'),
  (107, 'c'),
  (107, 'c'),
  (108, 'c'),
  (108, 'c'),
  (108, 'c'),
  (109, 'c'),
  (109, 'c'),
  (109, 'c'),
  (110, 'c'),
  (110, 'c'),
  (110, 'c'),
  (110, 'c'),
  (110, 'c'),
  (111, 'c'),
  (111, 'c'),
  (111, 'c'),
  (111, 'c'),
  (158, 'c'),
  (195, 'c'),
  (195, 'c'),
  (2